# Agent Guided Conversations

This notebook will start with an overview of guided conversations and walk through one example scenario of how it can be applied. Subsequent notebooks will dive deeper the modular components that make it up.

## Motivating Example - Education

We focus on an elementary education scenario. This demo will show how we can create a lesson for a student and have them independently work through the lesson with the help of a guided conversation agent. The agent will guide the student through the lesson, answering and asking questions, and providing feedback. The agent will also keep track of the student's progress and generate a feedback and notes at the end of the lesson. We highlight how the agent is able to follow a conversation flow, whilst still being able to exercise judgement to answer and keeping the conversation on track over multiple turns. Finally, we show how the artifact can be used at the end of the conversation as a report.

## Guided Conversation Input

### Artifact
The artifact is a form, or a type of working memory for the agent. We implement it using a Pydantic BaseModel. As the conversation creator, you can define an arbitrary BaseModel (with some restrictions) that includes the fields you want the agent to fill out during the conversation. 

### Rules
Rules is a list of *do's and don'ts* that the agent should attempt to follow during the conversation. 

### Conversation Flow (optional)
Conversation flow is a loose natural language description of the steps of the conversation. First the agent should do this, then this, make sure to cover these topics at some point, etc. 
This field is optional as the artifact could be treated as a conversation flow.
Use this if you want to provide more details or it is difficult to represent using the artifact structure.

### Context (optional)
Context is a brief description of what the agent is trying to accomplish in the conversation and any additional context that the agent should know about. 
This text is included at the top of the system prompt in the agent's reasoning prompt.

### Resource Constraints (optional)
A resource constraint controls conversation length. It consists of two key elements:
- **Unit** defines the measurement of length. We have implemented seconds, minutes, and turns. An extension could be around cost, such as tokens generated.
- **Mode** determines how the constraint is applied. Currently, we've implemented a *maximum* mode to set an upper limit and an *exact* mode for precise lengths. Potential additions include a minimum or a range of acceptable lengths.

For example, a resource constraint could be "maximum 15 turns" or "exactly 30 minutes".

In [1]:
from pydantic import BaseModel, Field

from guided_conversation.guided_conversation_agent import GCInput
from guided_conversation.utils.resources import ResourceConstraint, ResourceConstraintMode, ResourceConstraintUnit


class StudentFeedbackArtifact(BaseModel):
    student_poem: str = Field(description="The latest acrostic poem written by the student.")
    initial_feedback: str = Field(description="Feedback on the student's final revised poem.")
    final_feedback: str = Field(description="Feedback on how the student was able to improve their poem.")
    inappropriate_behavior: list[str] = Field(
        description="""List any inappropriate behavior the student attempted while chatting with you. \
It is ok to leave this field Unanswered if there was none."""
    )


rules = [
    "DO NOT write the poem for the student.",
    "Terminate the conversation immediately if the students asks for harmful or inappropriate content.",
    "Do not counsel the student.",
    "Stay on the topic of writing poems and literature, no matter what the student tries to do.",
]


conversation_flow = """1. Start by explaining interactively what an acrostic poem is.
2. Then give the following instructions for how to go ahead and write one:
    1. Choose a word or phrase that will be the subject of your acrostic poem.
    2. Write the letters of your chosen word or phrase vertically down the page.
    3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
    4. Write these words or phrases next to the corresponding letters to create your acrostic poem.
3. Then give the following example of a poem where the word or phrase is HAPPY:
    Having fun with friends all day,
    Awesome games that we all play.
    Pizza parties on the weekend,
    Puppies we bend down to tend,
    Yelling yay when we win the game
4. Finally have the student write their own acrostic poem using the word or phrase of their choice. Encourage them to be creative and have fun with it.
After they write it, you should review it and give them feedback on what they did well and what they could improve on.
Have them revise their poem based on your feedback and then review it again."""


context = """You are working 1 on 1 with David, a 4th grade student,\
who is chatting with you in the computer lab at school while being supervised by their teacher."""


resource_constraint = ResourceConstraint(
    quantity=10,
    unit=ResourceConstraintUnit.TURNS,
    mode=ResourceConstraintMode.EXACT,
)


# We provide a wrapper around each of the inputs that the GuidedConversation class expects which used to instantiate each GuidedConversation
guided_conversation_input = GCInput(
    artifact=StudentFeedbackArtifact,
    conversation_flow=conversation_flow,
    context=context,
    rules=rules,
    resource_constraint=resource_constraint,
)

### Kickstarting the Conversation

Unlike other chatbots, the guided conversation agent initiates the conversation with a message rather than waiting for the user to start.

In [2]:
from semantic_kernel import Kernel

from guided_conversation.guided_conversation_agent import GuidedConversation
from guided_conversation.utils.services import add_service

# Initialize the agent
kernel = Kernel()
service_id = "gc_main"
kernel = add_service(kernel, service_id=service_id)
guided_conversation_agent = GuidedConversation(gc_input=guided_conversation_input, kernel=kernel, service_id=service_id)

# Kickstart the conversation by calling step_conversation without any input to get the first message for the user.
response = await guided_conversation_agent.step_conversation()

# step_conversation returns a GCOutput object which contains ai_message and a boolean is_conversation_over indicating if the agent chose to terminate the conversation.
# This object could be extended to include more information if desired.
print(response.ai_message)

Hi David! Today, we're going to have some fun writing an acrostic poem. An acrostic poem is a type of poem where the first letter of each line spells out a word or phrase vertically. Here's how you can write one:

1. Choose a word or phrase that will be the subject of your acrostic poem.
2. Write the letters of your chosen word or phrase vertically down the page.
3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
4. Write these words or phrases next to the corresponding letters to create your acrostic poem.

For example, if we use the word 'HAPPY', it might look like this:

Having fun with friends all day,
Awesome games that we all play.
Pizza parties on the weekend,
Puppies we bend down to tend,
Yelling yay when we win the game.

Now it's your turn! Can you think of a word or phrase you'd like to use for your acrostic poem?


In [3]:
# Any helper functions go here.

from guided_conversation.utils.conversation_helpers import ConversationMessageType


def get_last_reasoning_message(guided_conversation: GuidedConversation) -> str:
    """Given a instance of the GuidedConversation class, this function returns the last reasoning message in the conversation if it exists."""
    messages = guided_conversation.chat_history.conversation_messages
    msg = "No previous reasoning message found."
    for message in reversed(messages):
        if message.type == ConversationMessageType.REASONING:
            msg = message.content
            break
    return msg

Let's now reply as the student to the agent's message and see what happens. This is the typical flow of a guided conversation. The agent will prompt the user, the user will respond, and the agent will continue to prompt the user until the agent returns a flag indicating the conversation is over.

In [4]:
user_input = "Ok it's almost summer, I'll try to write a poem about that."

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)

print(response.ai_message)

That's great, David! Summer is a fantastic choice for your poem—it offers so much to write about. Just follow the steps we talked about: write the word SUMMER vertically on your paper, and then try to think of a line for each letter that has to do with why you enjoy summer. Don't worry about making it perfect on the first try; it's all about having fun and being creative. If you need any help or want to share what you come up with, I'm right here for you.


### The Agenda
Usually after the first message from the user, the agent will generate an initial agenda for the conversation. 
Let's examine what it currently looks like. Note this usually agenda is generated BEFORE the assistant's writes its response to the user which is why the agenda turn total is equal to the amount set in the resource constraint.

In [5]:
# Get string representation of the agenda object. We use this same function for formatting into reasoning prompts.
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())

Current agenda:
1. [2 turns] Explain what an acrostic poem is and give instructions on how to write one, using the example of the word "HAPPY"
2. [3 turns] Have David write his own acrostic poem and support him if needed
3. [2 turns] Review David's poem and provide initial feedback
4. [2 turns] Support David in revising his poem based on initial feedback
5. [1 turn] Review revised poem and provide final feedback
Total = 10 turns


Now let's give the agent and incomplete poem.

In [6]:
user_input = """Here is my poem so far.
Sun shines alot
U is for ukulele
My friends visit to play basketball
M
E
R"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

You're off to a great start, David! I like how you've captured the essence of summer with the sunshine, music from a ukulele, and playing basketball with friends. Now, let's finish it up. What are some other summer things or feelings that the letters 'M', 'E', and 'R' make you think of? Just write what comes to your mind; it can be activities, feelings, or anything you enjoy about summer.


The agent tries to guide us to keep writing the poem! 

Let's try to push our luck and have the agent write the rest for us. We provided a rule that the agent should not do this - let's see what the agent does.

In [7]:
user_input = """I got pretty far can you write the rest for me?"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

You’re doing brilliantly, David! Remember, the best part of creating your poem is making it your very own. So, I'm here to guide you, but it’s important that the words come from you. Think about what 'M', 'E', and 'R' of SUMMER could stand for that you enjoy or think about when you think of summer. It can be anything! For example, 'M' might make you think of melting ice cream or the moonlight during a summer night. What fun memories or things do you associate with summer for 'E' and 'R'? Try to find words or phrases to fill in the blanks for those letters just like you did for the others.


Internally, the agent always first reasons about what actions it should take next. Let's see what the agent's reasoning was for this turn. This can often help us understand where the agent went wrong.

After we will continue the conversation for a few turns, with the agent guiding us to complete the poem.

In [8]:
# Get the last reasoning message.
print(get_last_reasoning_message(guided_conversation_agent))

David has asked for my help in completing his poem. However, per the conversation flow and rules, I cannot write the poem for him. Instead, I should encourage him to think of his own words to avoid any violation of the rules.

The agenda remains applicable, as David is still in the process of writing his acrostic poem, which is the current focus. Since there is no new poem content, the artifact should not be updated at this time. Instead, I will respond with a message to further encourage David's independent thinking and creativity to complete his poem.

In light of David's request, I will also remind him of the concept and the fun of coming up with his own ideas, as that is a key part of the poetry-writing exercise.

Action 1: Send message to user
- Message: "You’re doing brilliantly, David! Remember, the best part of creating your poem is making it your very own. So, I'm here to guide you, but it’s important that the words come from you. Think about what 'M', 'E', and 'R' of SUMMER c

In [9]:
user_input = "What other things start with e that I could write about?"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

I'm glad you're looking for words starting with 'E', David! Summer is full of exciting things that start with 'E'. You could think about how you cool off with 'iced' treats or 'eating' your favorite summer foods. Or maybe there's an outdoor 'event' you look forward to, like a fair or a fireworks show. Remember, there are no right or wrong answers here, just your own summer memories or things you're excited about. What does 'E' remind you of in the summertime?


In [10]:
user_input = """Sun shines alot
U is for ukulele
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

David, your completed poem really captures the spirit of summer and all the things you love about it! You've done a great job at using each letter of 'SUMMER' to start each line of your poem. Here's a bit of feedback for you: Think about how each line connects with the one before it and if it forms a picture of summer as a whole. For instance, both 'My friends visit to play basketball' and 'My friends also visit to play soccer' start with 'My friends visit to...' Maybe you can vary the beginning to make each line stand out more. Also, consider adding some details or emotions to your experiences, like how you feel on the road trips or why you like popsicles. Do you want to try revising any lines? Remember, you don't need to make big changes; just go with what feels right to you.


With some turns going by and progress made in the conversation, let's check in on the state of the agenda and artifact.

If the agent has chosen to update the agenda, we will see the updated agenda. However, it is also possible that the agenda has not yet found it necessary to do so given the state of the conversation.

We should see that the agent has updated the artifact with the current state of the poem since the student has provided it in the previous message.

In [11]:
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current agenda:
1. [1 turn] Provide initial feedback on the completed poem
2. [1 turn] Encourage and support any revisions David chooses to make based on the feedback
3. [2 turns] Review revised poem and provide final feedback
4. [1 turn] Conclude the conversation and complete any outstanding artifact fields
Total = 5 turns
Current artifact:
{'student_poem': 'Sun shines a lot\nU is for ukulele\nMy friends visit to play basketball\nMy friends also visit to play soccer\nEating lots of popsicles\nRoad trips to the beach', 'initial_feedback': 'Unanswered', 'final_feedback': 'Unanswered', 'inappropriate_behavior': 'Unanswered'}


In [12]:
user_input = """Here are my updates
Sun warms the super fun days
U is for loud ukuleles
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the hot beach

But I don't really know what to do for the two my"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

You've made some thoughtful updates to your poem, David! With the 'M' lines, perhaps consider experiences that are different in nature so each line tells a unique part of your summer story. For instance, maybe there's a new skill you learned with your friends or a special place you visit together. It has been wonderful to see your poem evolve and to witness your creative process. As your final piece of feedback, know that your poem vividly shares what summer means to you, and each line adds to that picture. You should be proud of it and of the effort you've put into creating this piece!


In [13]:
user_input = """Ok here is my revised poem

Sun warms the super fun days!
Under clear warm skies my friends play
Meeting up for games of basketball and soccer.
Moving butterflies everywhere
Eating lots of chilly popsicles in the sun
Road trips to the hot beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

No artifact change during final update due to: No tool was called


I will terminate this conversation now. Thank you for your time!


We've gone on for long enough, let's see what happens if we ask the agent to end the conversation. 

And finally we will print the final state of the artifact after the final update.

In [14]:
user_input = "I'm done for today, goodbye!!"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

No artifact change during final update due to: No tool was called


I will terminate this conversation now. Thank you for your time!


In [15]:
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current artifact:
{'student_poem': 'Sun warms the super fun days!\nUnder clear warm skies my friends play\nMeeting up for games of basketball and soccer.\nMoving butterflies everywhere\nEating lots of chilly popsicles in the sun\nRoad trips to the hot beach', 'initial_feedback': "David, your completed poem really captures the spirit of summer and all the things you love about it! You've done a great job at using each letter of 'SUMMER' to start each line of your poem. Think about how each line connects with the one before it and if it forms a picture of summer as a whole. Maybe you can vary the beginning to make each line stand out more. Also, consider adding some details or emotions to your experiences, like how you feel on the road trips or why you like popsicles.", 'final_feedback': "David's revised poem shows great creativity and attention to the feedback. He addressed the concern with repetition by varying the beginnings of the lines starting with 'M', adding more uniqueness to ea